# Polar Diatomic Molecules 

## Introduction
The aim of this notebook is to simulate a polar molecule, hydrogen chloride (HCl), to get a feel for the general behavior of polar molecules under different conditions. The polarity comes from the differing charges in the molecule: as chloride is more electronegative than hydrogen, it will pull more strongly on their shared electron, causing a partial positive charge at the hydrogen and a partial negative charge on the chlorine. In liquid form, between about -85$^\circ$ and -114$^\circ$ C, HCl should then behave in much the same way as water (another polar molecule). 

## Setting up the simulation

As always, we first need to import some toolboxes to make our lives easier. 

In [1]:
import numpy as np 
import time #to measure time
import matplotlib.pyplot as plt
np.random.seed(0) #sets the seed for the simulation

Now, we define some constants for the model, including the [atomic mass unit](https://physics.nist.gov/cgi-bin/cuu/Value?ukg) (u), [elementary electric charge](https://physics.nist.gov/cgi-bin/cuu/Value?e) (e), [vacuum permittivity](https://physics.nist.gov/cgi-bin/cuu/Value?ep0), Coulomb constant, and the [Boltzmann Constant](https://physics.nist.gov/cgi-bin/cuu/Value?k)

In [2]:
u = 1.66053906660e-27 #atomic mass unit
e_charge =  1.602176634e-19 # Elementary charge (1 eV)
vac_perm = 8.8541878128e-12 #s^2*C^2*/m^3/kg
k_e = 1/(4*np.pi*vac_perm) # m^3*kg/s^2/C^2 Coulomb constant
k_boltz = 1.380649e-23 #Boltzmann constant

Next, we define some simulation parameters: the grid dimensions, the corresponding number of molecules in the simulation, and the number/length of the timesteps in the simulation.

In [3]:
# Simulation parameters
grid_dims = np.array([4,4]) # Determines the number of molecules in the simulation (x*y)
num_molecules = np.prod(grid_dims) #explicitly determine number of molecules in the simulation, here set to fill the grid
timesteps = 5000 #Number of timesteps to simulate
dt = 5e-17 #[s] Extremely low timestep

We will model the polar molecules as an electric dipole, essentially positive charge bound to a negative charge. We create a function that generates the starting positions of the HCl molecules in a square grid, while each molecule is rotated at some random angle (by choosing the x and y value for the hydrogen atoms from a normal distribution). The two atoms that make up each molecule are initialized in their equilibrium positions, one 'bond_length' away from each other. 

In [4]:
def make_liquid(num_molecules, dims, bond_length, grid_dims, side_lengths):
    """Function to generate the polar molecules on a square grid"""
    
    #pos lets one atom spawn a bond_length away from the other atom, in a uniformly distributed random direction
    pos = np.random.normal(0,1,size=(num_molecules,dims)) #create an array to hold the x and y values of the hydrogen atom positions, relative to the molecule's location
    r = np.linalg.norm(pos, axis = 1) #determine the length of the random spacing between the atoms
    
    pos = pos/r.reshape(num_molecules,1)*bond_length #scale the positions by the random spacing length to normalize
    """Note: reshape allows the array of shape (num_molecules, dims) to be
    divided by an array (tuple) with shape (num_molecules, )
    by giving it shape (num_molecules, 1), both the x and y component will then be divided by r. """
    
    molecule = np.zeros((num_molecules, dims)) #array to hold the positions of the different atoms
    
    #initialize the molecules in a square grid, with random rotation
    xx = np.linspace(-side_lengths[0]/2, side_lengths[0]/2, grid_dims[0]) #x dimension for meshgrid
    yy = np.linspace(-side_lengths[1]/2, side_lengths[1]/2, grid_dims[1]) #y dimension for meshgrid
    mesh_x, mesh_y = np.meshgrid(xx,yy) #creates two meshgrids 
    
    #ravel flattens the 2d array into a 1d array
    molecule[:,0] = np.ravel(mesh_x) # Sets the x-position for all the hydrogen atoms
    molecule[:,1] = np.ravel(mesh_y) # Sets the y-position for all the hydrogen atoms
    # The centre of the molecules will be in a grid formation
    hydrogen = molecule - pos/2 
    chlorine = molecule + pos/2
    
    return hydrogen, chlorine

## Parameters for simulating HCl

With our constant and simulation function defined, we need to define some more specific parameters that will allow us to simulate a specific hydrogen chloride system. These include the number of atoms in a single molecule, the number of spatial dimensions, and the parameters for the specific type of molecule we're dealing with (HCl in this example). Note that the simulation is currently written to be only in 2D, but could certainly be expanded to 3D.

In [5]:
num_atoms = 2 #number of atoms in one molecule
dims = 2 #number of spatial dimensions 
# (the rest of the code is not written to handle anything but two dimensions, though you could change that)

partial_charge = 0.1811*e_charge # The partial charges for the atoms in HCl
# https://chem.libretexts.org/Courses/Mount_Royal_University/Chem_1201/Unit_3%3A_Chemical_Bonding_I_-_Lewis_Theory/3.4%3A_Bond_Polarity
charge_H = partial_charge # Effective charge of the hydrogen
charge_Cl = -partial_charge # Effective charge of the chlorine
# Cl is more electronegative than H, so the electron is, on average, closer to the Cl

mass_H = 1.00794*u # Mass of hydrogen
mass_Cl= 35.4527*u # Mass of chlorine
#http://www.chemicalelements.com/elements

bond_length = 0.13e-9 #[m] the distance between H and Cl in HCl
# http://hyperphysics.phy-astr.gsu.edu/hbase/molecule/vibrot.html#c3
side_lengths = [2*grid_dims[0]*bond_length, 2*grid_dims[1]*bond_length]
# Determines the spacing of the molecules in x and y direction
k_spring = 481 #bond strength of HCl
#http://hyperphysics.phy-astr.gsu.edu/hbase/molecule/vibrot.html#c2

Finally, we need to make some numpy arrays, so we can fill and use them later. These will be useful for both graphing and speeding up calculations. Then, we call the make_liquid function to create the simulation.

In [6]:
# Initialize numpy arrays
position_H = np.zeros((timesteps, num_molecules, dims))
position_Cl = np.zeros((timesteps, num_molecules, dims))
matrix_shape = position_H.shape 
# The other arrays (vel, acc, etc.) should be the same shape as the position arrays
velocity_H = np.zeros(matrix_shape) # Velocity
velocity_Cl = np.zeros(matrix_shape) # Velocity
acceleration_H = np.zeros(matrix_shape) # Acceleration
acceleration_Cl = np.zeros(matrix_shape) # Acceleration
coulomb_acc_H = np.zeros(matrix_shape) # Intermolecular coulomb interaction
coulomb_acc_Cl = np.zeros(matrix_shape) # Intermolecular coulomb interaction
spring_acc_H = np.zeros(matrix_shape) # Intramolecular interaction
spring_acc_Cl = np.zeros(matrix_shape) # Intramolecular interaction

# Set the initial position for all the atoms
position_H[0], position_Cl[0] = make_liquid(num_molecules, dims, bond_length, grid_dims, side_lengths)

## Forces and integration loop

To model the bonding between to atoms, to create a molecule, a harmonic oscillator potential is used. The force is then simply calculated usig Hooke's law with the appropriate constants, found in [hyperphysics](http://hyperphysics.phy-astr.gsu.edu/hbase/molecule/vibrot.html). This force is given by

$$ F(x) = k(x-x_0) $$

where $k = k\_spring$ is the spring constant, $x = spring\_distance$ is the distance between the two relevant atoms, and $x_0 = bond\_length$ is the equilibrium distance for HCl. 

The second force, which to us is the most interesting, is Coulomb force. We get Coulomb forces between HCl molecules because the hydrogen and the chlorine have opposite partial charges. The forces, between atoms in different molecules, are then given by

$$ F(x) = k_e\frac{q_Hq_{Cl}}{x^2} $$

where $k_e$ is the Coulomb constant, while $q_H = +\delta$ and $q_{Cl} = -\delta$ are the charges of the two atoms, where $\delta = 0.1811 e$ is the partial charge. 

To integrate these forces we use the familiar Euler-Chromer method. 

In [7]:
timeit_start = time.time()
for t in range(timesteps-1):
    #-----------------------------------------------
    # Spring potential to simulate atom interactions in a molecule, the bond force, using for loops
    # This potential is a good approximations only when the atoms are close to the equilibrium, 
    # it does not allow the bondings to "break"
    for i in range(num_molecules):
        spring_difference = position_Cl[t,i] - position_H[t,i]
        # position of one atom minus the position of the other,
        # gives the vector from H to Cl
        spring_distance = np.linalg.norm(spring_difference)
        # Distance between the bound atoms
        spring_unitvector = spring_difference/spring_distance
        # Unit vector pointing from H to Cl (from Cl to H would then be its negative)
        spring_force = k_spring*(spring_distance - bond_length) 
        # Hooke's law, spring constant*distance from equilibrium
        spring_acc_H[t,i] = spring_unitvector*spring_force/mass_H #direction*force/mass
        spring_acc_Cl[t,i] = -spring_unitvector*spring_force/mass_Cl #direction*force/mass

    #-----------------------------------------------
    # Coulomb force between molecules
    for i in range(num_molecules):#num_molecules):
        coulomb_mask = np.arange(0,num_molecules)!=i #a mask to ignore atoms in the same molecule (same index)
        # Helps us in somewhat vectorizing the calculation
        
        coulomb_difference_H_H = position_H[t,i] - position_H[t,coulomb_mask] 
        #x and y difference between the hydrogen and other all other hydrogen atoms
        coulomb_difference_H_Cl = position_H[t,i] - position_Cl[t,coulomb_mask] 
        #x and y difference between the hydrogen and all chlorine atoms in other molecules
        coulomb_difference_Cl_H = position_Cl[t,i] - position_H[t,coulomb_mask] 
        #x and y difference between the chlorine and all hydrogen atoms in other molecules
        coulomb_difference_Cl_Cl = position_Cl[t,i] - position_Cl[t,coulomb_mask]
        #x and y difference between the hydrogen and other all other hydrogen atoms
        
        # Calculates the distance from one atom to num_molecules-1 other atoms
        coulomb_distance_H_H = np.linalg.norm(coulomb_difference_H_H, axis = 1) # The distances between the atoms
        coulomb_distance_H_Cl = np.linalg.norm(coulomb_difference_H_Cl, axis = 1)
        coulomb_distance_Cl_H = np.linalg.norm(coulomb_difference_Cl_H, axis = 1)
        coulomb_distance_Cl_Cl = np.linalg.norm(coulomb_difference_Cl_Cl, axis = 1)
        
        # Reshape distance from (num_molecules-1,) to (num_molecules-1, 1)
        # This gives us num_molecules-1 unit vectors from one atom to num_molecules-1 others
        coulomb_unitvector_H_H = coulomb_difference_H_H/coulomb_distance_H_H.reshape(-1,1)
        coulomb_unitvector_H_Cl = coulomb_difference_H_Cl/coulomb_distance_H_Cl.reshape(-1,1)
        coulomb_unitvector_Cl_H = coulomb_difference_Cl_H/coulomb_distance_Cl_H.reshape(-1,1)
        coulomb_unitvector_Cl_Cl = coulomb_difference_Cl_Cl/coulomb_distance_Cl_Cl.reshape(-1,1)
        
        # Calculate the force according to Coulomb's law
        coulomb_force_H_H = k_e*charge_H*charge_H*coulomb_unitvector_H_H/coulomb_distance_H_H.reshape(-1,1)**2
        coulomb_force_H_Cl = k_e*charge_H*charge_Cl*coulomb_unitvector_H_Cl/coulomb_distance_H_Cl.reshape(-1,1)**2
        coulomb_force_Cl_H = k_e*charge_Cl*charge_H*coulomb_unitvector_Cl_H/coulomb_distance_Cl_H.reshape(-1,1)**2
        coulomb_force_Cl_Cl = k_e*charge_Cl*charge_Cl*coulomb_unitvector_Cl_Cl/coulomb_distance_Cl_Cl.reshape(-1,1)**2
        
        # Sum together the forces from the different molecules acting upon atom i, and divide by mass to get acc
        coulomb_acc_H[t,i] = (np.sum(coulomb_force_H_H, axis = 0)\
                           + np.sum(coulomb_force_H_Cl, axis = 0))/mass_H
        coulomb_acc_Cl[t,i] = (np.sum(coulomb_force_Cl_H, axis = 0)\
                            + np.sum(coulomb_force_Cl_Cl, axis = 0))/mass_Cl
        
    #----------------------------------------------
    # Sum the accelerations
    acceleration_H[t] = coulomb_acc_H[t] + spring_acc_H[t]
    acceleration_Cl[t] = coulomb_acc_Cl[t] + spring_acc_Cl[t]
    
    # Euler Chromer
    velocity_H[t+1] = velocity_H[t] + acceleration_H[t]*dt
    velocity_Cl[t+1] = velocity_Cl[t] + acceleration_Cl[t]*dt
    position_H[t+1] = position_H[t] + velocity_H[t+1]*dt
    position_Cl[t+1] = position_Cl[t] + velocity_Cl[t+1]*dt
    
    if t!=0 and t%int(timesteps/20) == 0: # Every long for-loop needs some kind of "loading bar"
        print(t, 'of', timesteps)
timeit_stop = time.time()
print('It took %.1f seconds to simulate %i molecules for %i timesteps' \
      %(timeit_stop-timeit_start, num_molecules, timesteps))

250 of 5000
500 of 5000
750 of 5000
1000 of 5000
1250 of 5000
1500 of 5000
1750 of 5000
2000 of 5000
2250 of 5000
2500 of 5000
2750 of 5000
3000 of 5000
3250 of 5000
3500 of 5000
3750 of 5000
4000 of 5000
4250 of 5000
4500 of 5000
4750 of 5000
It took 9.5 seconds to simulate 16 molecules for 5000 timesteps


Here we graph the simulation, to see how the molecules behave. 

In [8]:
plt.subplot(121) #Plots the initial positions of the molecules
plt.gca().set_aspect('equal', adjustable='box')
plt.plot(position_H[0,:,0], position_H[0,:,1], '+r', lw=2)
plt.plot(position_Cl[0,:,0], position_Cl[0,:,1], 'b', marker = 1, linestyle = 'None')

plt.subplot(122) # Plots the final position of the molecules
plt.gca().set_aspect('equal', adjustable='box')
plt.plot(position_H[-1,:,0], position_H[-1,:,1], '+r', lw=2)
plt.plot(position_Cl[-1,:,0], position_Cl[-1,:,1], 'b', marker = 1, linestyle = 'None')

plt.show()

## Additional questions you might investigate 

1. Can you attract the liquid? 
    * https://www.youtube.com/watch?v=yqU8vVPX0Cw 
2. How would the molecules line up with a large charge in the centre? What physical situation would this simulate?
3. Is there some way to model surface tension, like water on a teflon pan? 
4. How would you modify this program to simulate water, instead of HCl?
5. Currently, the only intermolecular forces are caused by Coulomb forces. However, this might be a problem if the atoms get too close to one another. What would you add to this simulation to make that behavior more realistic? What effect does this have on the simulation behavior?
7. How would the polar molecules react to a homogeneous or inhomogenous electric field? What physical situation would this simulate?
8. Is Euler-Chromer the optimal integration method? Why/why not? What changes with other integration methods?

Note that one of the first things you might try to do is to animate the motion of the molecules, since it is difficult to envision how the molecules behave during the simulation without being able to see them moving. Matplotlib has an animation library that works well without too much coding required.

*(These are just meant to be suggestions—feel free to investigate any question you find interesting!)*

